In [8]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model_neo = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

# Encode the input text
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate text
outputs = model.generate(inputs["input_ids"], max_length=50, do_sample=True)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text (GPT-Neo):", generated_text)


Generated text (GPT-Neo): Once upon a time var% this Miss hacks-ophob%�j sentencing- valvej�j sentencing-592j sentencing- valvej�� sentencing Adrow 37 membersues sentencing Adrow 37izeues this spec-row 37 merigun


In [9]:
from transformers import OPTForCausalLM, GPT2Tokenizer

# Load the tokenizer and model for OPT-125M
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-125m")
model_sigma = OPTForCausalLM.from_pretrained("facebook/opt-125m")

# Encode the input text
input_text = "Once upon a time"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate text
outputs = model.generate(inputs["input_ids"], max_length=50, do_sample=True)

# Decode and print the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated text (OPT-125M):", generated_text)



Generated text (OPT-125M): Once upon a time I saw 3 girls who didn't want to be part of either of my "team" and I've never regretted it. I've learned the hard way it's possible to have a crush on some of these people...and


In [63]:
import math 
from einops import einsum 
import torch


In [64]:
def evaluate_model(model, tokenizer, dataset):
    model.eval()
    total_loss = 0
    total_tokens = 0
    with torch.no_grad():
        for item in dataset:
            inputs = tokenizer(item['text'], return_tensors='pt', truncation=True, max_length=512)
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']
            
            
            masked_input = input_ids.clone()
            mask = torch.rand(masked_input.shape) < 0.15 
            masked_input[mask] = tokenizer.mask_token_id
            
            outputs = model(masked_input, attention_mask=attention_mask, labels=input_ids)
            
            
            if isinstance(outputs, tuple):
                loss = outputs[0]
            else:
                loss = outputs.loss
            
            total_loss += loss.item() * input_ids.numel()
            total_tokens += input_ids.numel()
    
    perplexity = math.exp(total_loss / total_tokens)
    return perplexity

In [11]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np

# Load models and tokenizers
gpt_neo_model = AutoModel.from_pretrained("EleutherAI/gpt-neo-125M")
opt_model = AutoModel.from_pretrained("facebook/opt-125m")
gpt_neo_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
opt_tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

gpt_neo_vocab = gpt_neo_tokenizer.get_vocab()
opt_vocab = opt_tokenizer.get_vocab()

shared_tokens = set(gpt_neo_vocab.keys()) & set(opt_vocab.keys())
print(f"Number of shared tokens: {len(shared_tokens)}")

gpt_neo_embeddings = gpt_neo_model.wte.weight.detach().numpy()
opt_embeddings = opt_model.get_input_embeddings().weight.detach().numpy()

shared_gpt_neo_embeds = []
shared_opt_embeds = []

for token in shared_tokens:
    gpt_neo_idx = gpt_neo_vocab[token]
    opt_idx = opt_vocab[token]
    shared_gpt_neo_embeds.append(gpt_neo_embeddings[gpt_neo_idx])
    shared_opt_embeds.append(opt_embeddings[opt_idx])

shared_gpt_neo_embeds = np.array(shared_gpt_neo_embeds)
shared_opt_embeds = np.array(shared_opt_embeds)

W = np.linalg.lstsq(shared_gpt_neo_embeds, shared_opt_embeds, rcond=None)[0]
W_1 = np.linalg.lstsq(shared_opt_embeds, shared_gpt_neo_embeds, rcond=None)[0]

print("Shape of transformation matrix W:", W.shape)

transformed_gpt_neo_embeds = np.dot(shared_gpt_neo_embeds, W)
mse = np.mean((transformed_gpt_neo_embeds - shared_opt_embeds)**2)
print("Mean Squared Error:", mse)

from sklearn.metrics.pairwise import cosine_similarity

cos_sims = [cosine_similarity(transformed_gpt_neo_embeds[i].reshape(1, -1), 
                              shared_opt_embeds[i].reshape(1, -1))[0][0] 
            for i in range(len(shared_tokens))]

average_cosine_similarity = np.mean(cos_sims)
print("Average Cosine Similarity:", average_cosine_similarity)

print("\nExamples of transformed embeddings:")
for i, token in enumerate(list(shared_tokens)[:5]):
    print(f"\nToken: {token}")
    print(f"Original GPT-Neo embedding: {shared_gpt_neo_embeds[i][:5]}...")
    print(f"Transformed GPT-Neo embedding: {transformed_gpt_neo_embeds[i][:5]}...")
    print(f"Original OPT embedding: {shared_opt_embeds[i][:5]}...")
    print(f"Cosine Similarity: {cos_sims[i]}")

Number of shared tokens: 50257
Shape of transformation matrix W: (768, 768)
Mean Squared Error: 0.001723926
Average Cosine Similarity: 0.73685193

Examples of transformed embeddings:

Token: Ġmanipulate
Original GPT-Neo embedding: [ 0.60546875 -0.81640625  0.29101562 -0.17480469 -0.34570312]...
Transformed GPT-Neo embedding: [ 0.03076771 -0.03459737  0.01216576  0.00620542  0.03119646]...
Original OPT embedding: [ 0.0383606  -0.05307007  0.00072908 -0.00559616 -0.046875  ]...
Cosine Similarity: 0.847366213798523

Token: ĠRussian
Original GPT-Neo embedding: [ 0.3671875  -0.65234375  0.33007812 -0.16210938 -0.22558594]...
Transformed GPT-Neo embedding: [ 0.03383368 -0.03290333 -0.05100903  0.07383785  0.02786811]...
Original OPT embedding: [ 0.04745483 -0.03747559 -0.08984375  0.11096191  0.105896  ]...
Cosine Similarity: 0.7892986536026001

Token: Ġinvaders
Original GPT-Neo embedding: [ 0.67578125 -1.0546875   0.59375    -0.37304688 -0.30273438]...
Transformed GPT-Neo embedding: [ 0.046

In [46]:
def get_residual_stream_neo(model, input_ids):

    input_embeds = model.wte(input_ids)
    temp = input_embeds
    for block in model.h:
        block_output = block(temp)
        temp = block_output[0]
    return temp


In [44]:
def get_residual_stream_opt(model, input_ids):
    input_embeds = model.embed_tokens(input_ids)
    temp = input_embeds
    for layer in model.layers:
        block_output = layer(temp)
        temp = block_output[0]
    return temp


In [57]:
import torch
import torch.nn as nn
from transformers import GPTNeoForCausalLM, OPTForCausalLM, GPT2Tokenizer
from tqdm import tqdm

class HybridModel(nn.Module):
    def __init__(self, gpt_neo_model, opt_model, W, W_inv):
        super(HybridModel, self).__init__()
        self.gpt_neo = gpt_neo_model
        self.opt = opt_model
        self.W = nn.Parameter(torch.tensor(W, dtype=torch.float32))
        self.W_inv = nn.Parameter(torch.tensor(W_inv, dtype=torch.float32))

    def forward(self, input_ids, attention_mask=None):
        # Get GPT-Neo embeddings
        gpt_neo_embeds = self.gpt_neo.transformer.wte(input_ids)
        
        # Transform to OPT space
        opt_space = torch.einsum('ij,bsj->bsi', self.W_inv, gpt_neo_embeds)
        
        # Pass through OPT model
        opt_outputs = self.opt(inputs_embeds=opt_space, attention_mask=attention_mask)
        
        # Transform back to GPT-Neo space
        gpt_neo_space = torch.einsum('ij,bsj->bsi', self.W, opt_outputs)
        
        # Final prediction using GPT-Neo's language model head
        logits = self.gpt_neo.lm_head(gpt_neo_space)
        
        return logits

def evaluate_model(model, tokenizer, dataset, device):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for item in tqdm(dataset):
            inputs = tokenizer(item['text'], return_tensors='pt', truncation=True, max_length=512, padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            labels = inputs['input_ids'].clone()
            
            # Create input with some masked tokens
            mask = torch.rand(inputs['input_ids'].shape) < 0.15
            inputs['input_ids'][mask] = tokenizer.mask_token_id
            
            outputs = model(**inputs)
            
            loss = nn.CrossEntropyLoss()(outputs.view(-1, outputs.size(-1)), labels.view(-1))
            
            total_loss += loss.item() * labels.numel()
            total_tokens += labels.numel()
    
    perplexity = torch.exp(torch.tensor(total_loss / total_tokens))
    return perplexity.item()

# Load models and tokenizer
gpt_neo_model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")
opt_model = OPTForCausalLM.from_pretrained("facebook/opt-125m")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

# Initialize W and W_inv (you would typically learn these)
W = torch.randn(768, 768)
W_inv = torch.linalg.inv(W)

# Create hybrid model
hybrid_model = HybridModel(gpt_neo_model, opt_model, W, W_inv)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hybrid_model.to(device)

# Example inference
text = "The cat sat on the"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = hybrid_model(**inputs)
next_token = outputs.argmax(dim=-1)
generated_text = tokenizer.decode(next_token[0])
print(f"Generated text: {text} {generated_text}")

# Example evaluation (you would need to provide your own dataset)
# dataset = load_your_dataset()
# perplexity = evaluate_model(hybrid_model, tokenizer, dataset, device)
# print(f"Model perplexity: {perplexity}")

/var/folders/x1/2kkd63rx6sj_6j0_l82_lbj40000gn/T/ipykernel_19566/2569182483.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W = nn.Parameter(torch.tensor(W, dtype=torch.float32))
/var/folders/x1/2kkd63rx6sj_6j0_l82_lbj40000gn/T/ipykernel_19566/2569182483.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W_inv = nn.Parameter(torch.tensor(W_inv, dtype=torch.float32))


TypeError: expected Tensor as element 1 in argument 1, but got CausalLMOutputWithPast

In [141]:
import torch
import torch.nn as nn
from transformers import GPTNeoForCausalLM, OPTForCausalLM, GPT2Tokenizer
from tqdm import tqdm

class HybridModel(nn.Module):
    def __init__(self, gpt_neo_model, opt_model, W, W_inv):
        super(HybridModel, self).__init__()
        self.gpt_neo = gpt_neo_model
        self.opt = opt_model
        self.W = nn.Parameter(torch.tensor(W, dtype=torch.float32))
        self.W_inv = nn.Parameter(torch.tensor(W_inv, dtype=torch.float32))

    def forward(self, input_ids, attention_mask=None):
        # Get GPT-Neo embeddings
        
        gpt_neo_embeds = self.gpt_neo.transformer.wte(torch.tensor(input_ids, dtype = int))
        
        # Transform to OPT space
        opt_space = torch.einsum('ij,bsj->bsi', self.W_inv, gpt_neo_embeds)
        
        # Pass through OPT model
        opt_outputs = self.opt(inputs_embeds=opt_space, attention_mask=attention_mask)
        
        # Transform back to GPT-Neo space
        gpt_neo_space = torch.einsum('ij,bsj->bsi', self.W, opt_outputs.hidden_states[-1])
        
        # Final prediction using GPT-Neo's language model head
        logits = self.gpt_neo.lm_head(gpt_neo_space)
        
        return logits

def evaluate_model(model, tokenizer, dataset, device):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for item in tqdm(dataset):
            inputs = tokenizer(item['text'], return_tensors='pt', truncation=True, max_length=512, padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            labels = inputs['input_ids'].clone()
            print(f"Lables = {labels} with shape {labels.shape}")
            
            # Create input with some masked tokens
            mask = torch.rand(inputs['input_ids'].shape) < 0.15
            inputs['input_ids'][mask] = tokenizer.pad_token_id  # Use pad token ID for masking
            
            outputs = model(**inputs)
            print(f"Outputs = {outputs} with shape = {outputs.shape}")
            
            loss = nn.CrossEntropyLoss()(outputs.view(-1, outputs.size(-1)), labels.view(-1))
            print(loss)
            total_loss += loss.item() * labels.numel()
            total_tokens += labels.numel()
            print(total_loss)
    perplexity = torch.exp(torch.tensor(total_loss / total_tokens))
    return perplexity.item()

# Load models and tokenizer
gpt_neo_model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M", output_hidden_states=True)
opt_model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_hidden_states=True)
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

# Initialize W and W_inv (you would typically learn these)
W = torch.randn(768, 768)
W_inv = torch.linalg.inv(W)

# Create hybrid model
hybrid_model = HybridModel(gpt_neo_model, opt_model, W, W_inv)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hybrid_model.to(device)

# Example inference
text = "The cat sat on the"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = hybrid_model(**inputs)
next_token = outputs.argmax(dim=-1)
generated_text = tokenizer.decode(next_token[0])
print(f"Generated text: {text} {generated_text}")

# Example evaluation (you would need to provide your own dataset)
# dataset = load_your_dataset()
# perplexity = evaluate_model(hybrid_model, tokenizer, dataset, device)
# print(f"Model perplexity: {perplexity}")



Generated text: The cat sat on the ��極��極��極��極��極


/var/folders/x1/2kkd63rx6sj_6j0_l82_lbj40000gn/T/ipykernel_19566/3880965412.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W = nn.Parameter(torch.tensor(W, dtype=torch.float32))
/var/folders/x1/2kkd63rx6sj_6j0_l82_lbj40000gn/T/ipykernel_19566/3880965412.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W_inv = nn.Parameter(torch.tensor(W_inv, dtype=torch.float32))
/var/folders/x1/2kkd63rx6sj_6j0_l82_lbj40000gn/T/ipykernel_19566/3880965412.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gpt_neo_embeds = self.gpt_neo

In [87]:
text = "The cat sat on the"
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = hybrid_model(**inputs)
next_token = outputs.argmax(dim=-1)
generated_text = tokenizer.decode(next_token[0])
print(f"Generated text: {text} {generated_text}")

Generated text: The cat sat on the  renegotireditaryreditaryreditary STATS


In [88]:
tokenizer.pad_token = tokenizer.eos_token

In [69]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")


subset_size = 1000  
dataset = dataset.select(range(subset_size))



In [92]:
test = torch.randn([1,5])

In [100]:
out = gpt_neo_model.transformer.wte(torch.tensor(test, dtype = int))

/var/folders/x1/2kkd63rx6sj_6j0_l82_lbj40000gn/T/ipykernel_19566/719630797.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = gpt_neo_model.transformer.wte(torch.tensor(test, dtype = int))


In [125]:
def evaluate_model(model, tokenizer, dataset, device):
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for item in tqdm(dataset):
            inputs = tokenizer(item['text'], return_tensors='pt', truncation=True, max_length=512, padding=True)
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            labels = inputs['input_ids'].clone()
            
            # Create input with some masked tokens
            mask = torch.rand(inputs['input_ids'].shape).to(device) < 0.15
            inputs['input_ids'][mask] = tokenizer.pad_token_id  # Use pad token ID for masking
            labels[mask] = -100  # Ignore the masked tokens in the loss
            
            outputs = model(**inputs)
            logits = outputs # Extract logits from model outputs
            
            loss = nn.CrossEntropyLoss()(logits.view(-1, logits.size(-1)), labels.view(-1))
            total_loss += loss.item() * labels.numel()
            total_tokens += (labels != -100).sum().item()
            print(total_loss)
    
    perplexity = torch.exp(torch.tensor(total_loss / total_tokens))
    return perplexity.item()

In [144]:
a = torch.randn([1, 8])
b = torch.randn([1, 8, 50257])

In [149]:
a = torch.randn([1, 8])
b = torch.randn([1, 8, 50257])
loss = nn.CrossEntropyLoss()(b.view(-1, outputs.size(-1)), a.view(-1))

RuntimeError: expected scalar type Long but found Float

In [142]:
tokenizer.pad_token = tokenizer.eos_token
perplexity = evaluate_model(hybrid_model, tokenizer, dataset, 'cpu')
print(f"Model perplexity: {perplexity}")

  0%|          | 0/1000 [00:00<?, ?it/s]/var/folders/x1/2kkd63rx6sj_6j0_l82_lbj40000gn/T/ipykernel_19566/3880965412.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gpt_neo_embeds = self.gpt_neo.transformer.wte(torch.tensor(input_ids, dtype = int))


Lables = tensor([], size=(1, 0)) with shape torch.Size([1, 0])
Outputs = tensor([], size=(1, 0, 50257)) with shape = torch.Size([1, 0, 50257])
tensor(nan)
nan
Lables = tensor([[ 796, 5199,  347, 2852,  353,  796,  220,  198]]) with shape torch.Size([1, 8])
Outputs = tensor([[[101.6950,  80.9773, 100.6089,  ..., 216.1597, 175.4110, 184.6033],
         [ 61.4785,  38.1162,  21.7412,  ..., 144.7068, 124.5626, 128.9748],
         [358.7428, 335.8615, 403.3374,  ..., 618.8394, 425.6807, 312.7843],
         ...,
         [ 84.0865,  55.7570,  61.6663,  ..., 189.7960, 157.3575, 144.5110],
         [268.3845, 263.5094, 342.4463,  ..., 392.5680, 286.3525, 286.3372],
         [ 84.8758,  52.6529,  60.2078,  ..., 183.4696, 153.7621, 141.7562]]]) with shape = torch.Size([1, 8, 50257])
tensor(569.4001)
nan
Lables = tensor([], size=(1, 0)) with shape torch.Size([1, 0])
Outputs = tensor([], size=(1, 0, 50257)) with shape = torch.Size([1, 0, 50257])
tensor(nan)
nan
Lables = tensor([[ 5199,   347,  285

  0%|          | 4/1000 [00:00<01:28, 11.22it/s]

Outputs = tensor([[[117.1242,  97.1438, 133.2613,  ..., 236.7221, 189.2747, 202.7625],
         [319.0764, 275.6877, 319.4665,  ..., 515.4469, 282.8264, 215.1320],
         [ 99.1492,  67.7402,  87.5453,  ..., 225.3622, 175.9447, 151.6208],
         ...,
         [ 82.1644,  46.3512,  56.9978,  ..., 192.3174, 151.8773, 136.1239],
         [209.9630, 180.6935, 255.2182,  ..., 367.6426, 182.5962, 169.7899],
         [ 80.0855,  44.9487,  53.7837,  ..., 186.9579, 148.3336, 134.2601]]]) with shape = torch.Size([1, 183, 50257])
tensor(510.5706)
nan
Lables = tensor([[  554,  4793,   837,   347,  2852,   353, 31636,  7848,   854,   680,
           707,   287,   262,   711, 47002,  3194,   416,  2940, 12552, 12639,
           764,   679,  4120,   319,   257,  4793,  4471,   286,   262,  5581,
          2168,   837, 28274,   837,  3940,   416,   257,  2597,   287,   262,
          4343, 21421,  3227,   286,  1374,   284, 19739,  7924,   416, 22568,
           494,   371, 49003,   764,  1374,   

  1%|          | 7/1000 [00:00<01:55,  8.60it/s]

...

Outputs = tensor([[[101.6951,  80.9775, 100.6090,  ..., 216.1599, 175.4112, 184.6035],
         [306.5869, 313.2511, 364.6369,  ..., 394.9286, 323.3239, 300.6893],
         [ 70.7351,  42.5686,  40.3791,  ..., 160.3911, 137.7240, 132.0894],
         ...,
         [ 78.2064,  44.3728,  54.5411,  ..., 180.0297, 146.0088, 132.8379],
         [197.7067, 171.9391, 257.6534,  ..., 301.2044, 181.1659, 179.6257],
         [ 76.2643,  42.7291,  51.2961,  ..., 174.9896, 142.8117, 130.9673]]]) with shape = torch.Size([1, 325, 50257])
tensor(487.8220)
nan
Lables = tensor([], size=(1, 0)) with shape torch.Size([1, 0])
Outputs = tensor([], size=(1, 0, 50257)) with shape = torch.Size([1, 0, 50257])
tensor(nan)
nan
Lables = tensor([[ 796,  796,  796, 2693, 4574,  796,  796,  796,  220,  198]]) with shape torch.Size([1, 10])
Outputs = tensor([[[101.6950,  80.9774, 100.6089,  ..., 216.1597, 175.4110, 184.6033],
         [ 62.0412,  38.5393,  22.1272,  ..., 145.7505, 125.2556, 128.8690],
         [ 65.59

 34%|███▍      | 340/1000 [00:47<02:02,  5.37it/s]

Outputs = tensor([[[121.6616, 109.6821, 146.6531,  ..., 244.3695, 190.6791, 217.3337],
         [ 66.4288,  42.6532,  30.5444,  ..., 153.9489, 131.7435, 132.9724],
         [283.5862, 251.6949, 343.7617,  ..., 493.6848, 345.8945, 307.0486],
         ...,
         [ 76.5099,  42.0621,  52.2704,  ..., 181.0714, 143.1637, 129.7016],
         [155.3022, 125.5675, 194.2445,  ..., 250.0874, 100.8905, 134.5676],
         [ 74.1639,  40.3316,  48.7359,  ..., 175.2664, 139.2401, 127.5011]]]) with shape = torch.Size([1, 121, 50257])
tensor(487.7423)
nan
Lables = tensor([], size=(1, 0)) with shape torch.Size([1, 0])
Outputs = tensor([], size=(1, 0, 50257)) with shape = torch.Size([1, 0, 50257])
tensor(nan)
nan
Lables = tensor([[ 796,  796, 5838,  796,  796,  220,  198]]) with shape torch.Size([1, 7])
Outputs = tensor([[[101.3162,  81.9670,  94.4325,  ..., 213.9025, 173.6245, 182.7834],
         [ 62.0818,  38.5437,  22.1607,  ..., 145.9296, 125.1568, 128.8382],
         [ 67.0207,  41.0719,  34.7

 34%|███▍      | 343/1000 [00:47<01:31,  7.21it/s]


KeyboardInterrupt: 

In [105]:
perplexity

nan